<a href="https://colab.research.google.com/github/louispaulet/pattern_mining_course/blob/main/PMSNA_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This version contains explanations about each and every step between the inital parsing of the data to the final result. There is no unique minepi() function here, rather a series of inputs and ouputs.

In [ ]:
import pandas as pd
df = pd.read_csv('YahooFinance.data', sep='	')
df

,ID_Sequence,ID_Time,Cotation
0,1,1,Microsoft=3
1,1,1,Boeing=0
2,1,1,Citigroup=3
3,1,1,Du_Pont=0
4,1,1,Exxon_Mobil=-2
...,...,...,...
18818,1,1255,American_Express=0
18819,1,1255,AT&T=0
18820,1,1255,Procter_&_Gamble=0
18821,1,1255,Verizon=0


In [ ]:
#the "ID_Sequence" column only contains the value "1" so we drop it

df = df.drop("ID_Sequence", axis=1)
df

,ID_Time,Cotation
0,1,Microsoft=3
1,1,Boeing=0
2,1,Citigroup=3
3,1,Du_Pont=0
4,1,Exxon_Mobil=-2
...,...,...
18818,1255,American_Express=0
18819,1255,AT&T=0
18820,1255,Procter_&_Gamble=0
18821,1255,Verizon=0


In [ ]:
#true cardinality of the event names
event_names = df.Cotation.unique().tolist()
event_names

['Microsoft=3',
 'Boeing=0',
 'Citigroup=3',
 'Du_Pont=0',
 'Exxon_Mobil=-2',
 'HP=3',
 'Coca_Cola=-1',
 'General_Motors=3',
 'Intel=3',
 'Altria=-3',
 'American_Express=3',
 'AT&T=1',
 'Procter_&_Gamble=-2',
 'Verizon=3',
 'Morgan_Chase=3',
 'Microsoft=1',
 'Boeing=-1',
 'Citigroup=1',
 'Du_Pont=1',
 'Exxon_Mobil=-1',
 'HP=1',
 'General_Motors=2',
 'Intel=-1',
 'Altria=-2',
 'American_Express=-2',
 'Verizon=1',
 'Morgan_Chase=1',
 'Boeing=-2',
 'Citigroup=-2',
 'Du_Pont=-1',
 'Exxon_Mobil=0',
 'HP=-3',
 'Coca_Cola=0',
 'General_Motors=-3',
 'Intel=-3',
 'Altria=0',
 'AT&T=-2',
 'Procter_&_Gamble=2',
 'Verizon=0',
 'Morgan_Chase=-3',
 'Microsoft=0',
 'Citigroup=0',
 'Coca_Cola=1',
 'General_Motors=-1',
 'Intel=0',
 'Altria=2',
 'American_Express=-3',
 'AT&T=0',
 'Procter_&_Gamble=1',
 'Morgan_Chase=0',
 'Citigroup=-1',
 'Du_Pont=-2',
 'HP=0',
 'AT&T=2',
 'Procter_&_Gamble=-1',
 'Morgan_Chase=-1',
 'Boeing=1',
 'Citigroup=2',
 'Intel=1',
 'Altria=1',
 'American_Express=0',
 'Morgan_Chas

An id-list is a list of all sequences that contain the event. We must create an id_list for every event_name

In [ ]:
id_lists = []
for event_name in event_names:
  id_lists.append((event_name, df[df.Cotation == event_name].ID_Time.tolist()))


len(id_lists)

105

For the next step, we create a second list of id_lists that doesn't contains the "0" variations (exclude the lack of variation).

In [ ]:
import re

zeroless_id_list = []

for id_list in id_lists:

  #exclude all values that contain "=0" string
  match = re.search("^.*(\=0)", id_list[0])

  if not match:
    zeroless_id_list.append(id_list)


# we can see that we removed 15 event_types from the id_lists,
# wich corresponds to the number of company names analysed
len(id_lists) , len(zeroless_id_list)

(105, 90)

We are only interested in frequent events with a frequency >= 50 

In [ ]:
frequent_id_list = []

for zeroless_id in zeroless_id_list:
  if len(zeroless_id[1]) >= 50:
    frequent_id_list.append(zeroless_id)

#compare the difference between frequent events and non-frequent
print(len(zeroless_id_list), len(frequent_id_list))

90 44


We want to have a size of events of 2 so we must merge event_types with sequences that have a gap below to min_gap constraint

In [ ]:
from itertools import permutations

#gather cardinality of frequent event_types
card_event_types = []
for frequent_id in frequent_id_list:
  card_event_types.append(frequent_id[0])

#list all pairs of even_types
pairs_event_types = list(permutations(card_event_types, 2))

#how many permutations of events (sequences) do we have ?
len(pairs_event_types)

1892

We build a dataframe that contains the permutations, the cardinalities, their head_frequencies, confidence, and recall

In [ ]:
df_event_types = pd.DataFrame(columns=["Xn", "Yn", "XYhead", "frequency", "confidence", "recall", "Xseq", "Yseq", "XYjoinSeq", "L", "hMeanConfRec"])
df_event_typesXY = pd.DataFrame(pairs_event_types, columns=["X", "Y"])
df_event_types = pd.concat([df_event_typesXY, df_event_types], axis=1)

df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Coca_Cola=-1,AT&T=1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Coca_Cola=-1,Microsoft=1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Coca_Cola=-1,Boeing=-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Coca_Cola=-1,Citigroup=1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1888,Morgan_Chase=-2,American_Express=2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1889,Morgan_Chase=-2,Boeing=2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1890,Morgan_Chase=-2,Altria=-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Then we add the cardinalities to the columns (as well as sequences for later)

In [ ]:
for frequent_id in frequent_id_list:
  df_event_types.loc[df_event_types.X == frequent_id[0], "Xn"] = len(frequent_id[1])
  df_event_types.loc[df_event_types.X == frequent_id[0], "Xseq"] = [frequent_id[1]]

  df_event_types.loc[df_event_types.Y == frequent_id[0], "Yn"] = len(frequent_id[1])
  df_event_types.loc[df_event_types.Y == frequent_id[0], "Yseq"] = [frequent_id[1]]

df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,184,61,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 11, 14, 30, 48, 55, 65, 68, 69, 73, 80, 10...",NaN,NaN,NaN
1,Coca_Cola=-1,AT&T=1,184,223,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 2, 18, 28, 29, 32, 33, 36, 37, 41, 43, 50,...",NaN,NaN,NaN
2,Coca_Cola=-1,Microsoft=1,184,218,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 3, 6, 21, 23, 36, 42, 44, 51, 62, 67, 72, ...",NaN,NaN,NaN
3,Coca_Cola=-1,Boeing=-1,184,269,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...",NaN,NaN,NaN
4,Coca_Cola=-1,Citigroup=1,184,221,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 9, 14, 15, 18, 23, 28, 48, 52, 62, 69, ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,57,79,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[13, 26, 27, 38, 46, 53, 77, 79, 84, 95, 98, 1...",NaN,NaN,NaN
1888,Morgan_Chase=-2,American_Express=2,57,60,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[14, 48, 50, 58, 80, 85, 123, 132, 186, 194, 1...",NaN,NaN,NaN
1889,Morgan_Chase=-2,Boeing=2,57,56,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[16, 20, 28, 33, 42, 44, 52, 57, 65, 94, 104, ...",NaN,NaN,NaN
1890,Morgan_Chase=-2,Altria=-1,57,182,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[17, 20, 36, 37, 43, 49, 50, 52, 54, 59, 62, 7...",NaN,NaN,NaN


We need to join Xseq and Yseq into XYjoinSeq with maxSpan param(we consider minGap = 1 and maxSpan = 2).

In [ ]:
def joinSeqs(Xseq, Yseq, maxSpan, minGap):
  joinSeq = []
  for Xseq_occurence in Xseq:
    for Yseq_occurrence in Yseq:
      #we want to perform a forward-join of at least Xseq_occurence+minGap
      if (Xseq_occurence + minGap > Yseq_occurrence):
        #case where the Xseq_occurrence is too low
        None;
      #we want to add the number to the join sequence if the maxSpan is respected
      elif (Yseq_occurrence - Xseq_occurence <= maxSpan):
        joinSeq.append((Xseq_occurence, Yseq_occurrence))
        #we add a break here because we don't care for non-minimal sub-occurrences
        break;
      else:
        #else we finished with this Xseq_occurrence and go to the next one
        break;
      
  #remove non-minimal included occurrences
  prec_occurrence = (-1, -1) #initialize with impossible values
  minimal_joinSeq = [] #final list to be returned

  for joinSeq_pair in joinSeq:
    #when a new pair is found, add it to the list
    if (prec_occurrence[1] != joinSeq_pair[1]):
      minimal_joinSeq.append(joinSeq_pair)
    #if the new pair has the same upper bound, replace first item
    else:
      #squash the previous pair with the current one
      minimal_joinSeq[len(minimal_joinSeq)-1] = joinSeq_pair

    prec_occurrence = joinSeq_pair

  return minimal_joinSeq

#test
test_x_seq = [1, 3, 4]
test_y_seq = [2, 3, 6]
maxSpan = 2
minGap = 1

joinSeqs(test_x_seq, test_y_seq, maxSpan, minGap)

[(1, 2), (4, 6)]

Apply the join between Xseq and Yseq in the dataframe to obtain XYjoinSeq

In [ ]:
maxSpan = 2
minGap = 1

df_event_types['XYjoinSeq'] = df_event_types.apply(lambda row: joinSeqs(row.Xseq, row.Yseq, maxSpan, minGap), axis=1)
df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,184,61,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 11, 14, 30, 48, 55, 65, 68, 69, 73, 80, 10...","[(12, 14), (29, 30), (47, 48), (53, 55), (63, ...",NaN,NaN
1,Coca_Cola=-1,AT&T=1,184,223,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 2, 18, 28, 29, 32, 33, 36, 37, 41, 43, 50,...","[(1, 2), (17, 18), (31, 32), (36, 37), (49, 50...",NaN,NaN
2,Coca_Cola=-1,Microsoft=1,184,218,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 3, 6, 21, 23, 36, 42, 44, 51, 62, 67, 72, ...","[(1, 2), (2, 3), (49, 51), (97, 99), (107, 108...",NaN,NaN
3,Coca_Cola=-1,Boeing=-1,184,269,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...","[(1, 2), (11, 12), (17, 18), (33, 34), (53, 54...",NaN,NaN
4,Coca_Cola=-1,Citigroup=1,184,221,NaN,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 9, 14, 15, 18, 23, 28, 48, 52, 62, 69, ...","[(1, 2), (8, 9), (12, 14), (17, 18), (47, 48),...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,57,79,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[13, 26, 27, 38, 46, 53, 77, 79, 84, 95, 98, 1...","[(24, 26), (258, 260), (323, 325), (336, 337),...",NaN,NaN
1888,Morgan_Chase=-2,American_Express=2,57,60,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[14, 48, 50, 58, 80, 85, 123, 132, 186, 194, 1...","[(130, 132), (228, 229), (283, 284), (333, 335...",NaN,NaN
1889,Morgan_Chase=-2,Boeing=2,57,56,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[16, 20, 28, 33, 42, 44, 52, 57, 65, 94, 104, ...","[(55, 57), (130, 132), (369, 370), (402, 404),...",NaN,NaN
1890,Morgan_Chase=-2,Altria=-1,57,182,NaN,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[17, 20, 36, 37, 43, 49, 50, 52, 54, 59, 62, 7...","[(81, 83), (107, 108), (130, 132), (170, 171),...",NaN,NaN


From the XYjoinSeq, we can count the number of pairs to obtain XYhead

In [ ]:
df_event_types['XYhead'] = df_event_types.apply(lambda row: len(row.XYjoinSeq), axis=1)
df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,184,61,20,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 11, 14, 30, 48, 55, 65, 68, 69, 73, 80, 10...","[(12, 14), (29, 30), (47, 48), (53, 55), (63, ...",NaN,NaN
1,Coca_Cola=-1,AT&T=1,184,223,62,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 2, 18, 28, 29, 32, 33, 36, 37, 41, 43, 50,...","[(1, 2), (17, 18), (31, 32), (36, 37), (49, 50...",NaN,NaN
2,Coca_Cola=-1,Microsoft=1,184,218,61,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 3, 6, 21, 23, 36, 42, 44, 51, 62, 67, 72, ...","[(1, 2), (2, 3), (49, 51), (97, 99), (107, 108...",NaN,NaN
3,Coca_Cola=-1,Boeing=-1,184,269,68,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...","[(1, 2), (11, 12), (17, 18), (33, 34), (53, 54...",NaN,NaN
4,Coca_Cola=-1,Citigroup=1,184,221,67,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 9, 14, 15, 18, 23, 28, 48, 52, 62, 69, ...","[(1, 2), (8, 9), (12, 14), (17, 18), (47, 48),...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,57,79,15,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[13, 26, 27, 38, 46, 53, 77, 79, 84, 95, 98, 1...","[(24, 26), (258, 260), (323, 325), (336, 337),...",NaN,NaN
1888,Morgan_Chase=-2,American_Express=2,57,60,14,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[14, 48, 50, 58, 80, 85, 123, 132, 186, 194, 1...","[(130, 132), (228, 229), (283, 284), (333, 335...",NaN,NaN
1889,Morgan_Chase=-2,Boeing=2,57,56,9,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[16, 20, 28, 33, 42, 44, 52, 57, 65, 94, 104, ...","[(55, 57), (130, 132), (369, 370), (402, 404),...",NaN,NaN
1890,Morgan_Chase=-2,Altria=-1,57,182,16,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[17, 20, 36, 37, 43, 49, 50, 52, 54, 59, 62, 7...","[(81, 83), (107, 108), (130, 132), (170, 171),...",NaN,NaN


One last information needed before computing the remaining stats is the length of the sequence.

In [ ]:
def length_of_seq(seq):
  length = 0
  for pair in seq:
    length+=pair[1] - pair[0]
  return length

df_event_types['L'] = df_event_types.apply(lambda row: length_of_seq(row.XYjoinSeq), axis=1)
df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,184,61,20,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 11, 14, 30, 48, 55, 65, 68, 69, 73, 80, 10...","[(12, 14), (29, 30), (47, 48), (53, 55), (63, ...",31,NaN
1,Coca_Cola=-1,AT&T=1,184,223,62,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 2, 18, 28, 29, 32, 33, 36, 37, 41, 43, 50,...","[(1, 2), (17, 18), (31, 32), (36, 37), (49, 50...",88,NaN
2,Coca_Cola=-1,Microsoft=1,184,218,61,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 3, 6, 21, 23, 36, 42, 44, 51, 62, 67, 72, ...","[(1, 2), (2, 3), (49, 51), (97, 99), (107, 108...",86,NaN
3,Coca_Cola=-1,Boeing=-1,184,269,68,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...","[(1, 2), (11, 12), (17, 18), (33, 34), (53, 54...",90,NaN
4,Coca_Cola=-1,Citigroup=1,184,221,67,NaN,NaN,NaN,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 9, 14, 15, 18, 23, 28, 48, 52, 62, 69, ...","[(1, 2), (8, 9), (12, 14), (17, 18), (47, 48),...",90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,57,79,15,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[13, 26, 27, 38, 46, 53, 77, 79, 84, 95, 98, 1...","[(24, 26), (258, 260), (323, 325), (336, 337),...",25,NaN
1888,Morgan_Chase=-2,American_Express=2,57,60,14,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[14, 48, 50, 58, 80, 85, 123, 132, 186, 194, 1...","[(130, 132), (228, 229), (283, 284), (333, 335...",22,NaN
1889,Morgan_Chase=-2,Boeing=2,57,56,9,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[16, 20, 28, 33, 42, 44, 52, 57, 65, 94, 104, ...","[(55, 57), (130, 132), (369, 370), (402, 404),...",15,NaN
1890,Morgan_Chase=-2,Altria=-1,57,182,16,NaN,NaN,NaN,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[17, 20, 36, 37, 43, 49, 50, 52, 54, 59, 62, 7...","[(81, 83), (107, 108), (130, 132), (170, 171),...",21,NaN


We define the missing statistics and apply them to the dataframe

In [ ]:
def frequency(head_ab, L):
  if (L == 0):
    return None
  return head_ab / L

def confidence(head_ab, na):
  if (na == 0):
    return None
  return head_ab / na

def recall(head_ab, nb):
  if (nb == 0):
    return None
  return head_ab / nb

df_event_types['frequency'] = df_event_types.apply(lambda row: frequency(row.XYhead, row.L), axis=1)
df_event_types['confidence'] = df_event_types.apply(lambda row: confidence(row.XYhead, row.Xn), axis=1)
df_event_types['recall'] = df_event_types.apply(lambda row: recall(row.XYhead, row.Yn), axis=1)

df_event_types

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
0,Coca_Cola=-1,Intel=3,184,61,20,0.645161,0.108696,0.327869,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 11, 14, 30, 48, 55, 65, 68, 69, 73, 80, 10...","[(12, 14), (29, 30), (47, 48), (53, 55), (63, ...",31,NaN
1,Coca_Cola=-1,AT&T=1,184,223,62,0.704545,0.336957,0.278027,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[1, 2, 18, 28, 29, 32, 33, 36, 37, 41, 43, 50,...","[(1, 2), (17, 18), (31, 32), (36, 37), (49, 50...",88,NaN
2,Coca_Cola=-1,Microsoft=1,184,218,61,0.709302,0.331522,0.279817,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 3, 6, 21, 23, 36, 42, 44, 51, 62, 67, 72, ...","[(1, 2), (2, 3), (49, 51), (97, 99), (107, 108...",86,NaN
3,Coca_Cola=-1,Boeing=-1,184,269,68,0.755556,0.369565,0.252788,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...","[(1, 2), (11, 12), (17, 18), (33, 34), (53, 54...",90,NaN
4,Coca_Cola=-1,Citigroup=1,184,221,67,0.744444,0.364130,0.303167,"[1, 2, 7, 8, 11, 12, 17, 18, 29, 31, 33, 36, 4...","[2, 7, 9, 14, 15, 18, 23, 28, 48, 52, 62, 69, ...","[(1, 2), (8, 9), (12, 14), (17, 18), (47, 48),...",90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,Morgan_Chase=-2,HP=-2,57,79,15,0.600000,0.263158,0.189873,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[13, 26, 27, 38, 46, 53, 77, 79, 84, 95, 98, 1...","[(24, 26), (258, 260), (323, 325), (336, 337),...",25,NaN
1888,Morgan_Chase=-2,American_Express=2,57,60,14,0.636364,0.245614,0.233333,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[14, 48, 50, 58, 80, 85, 123, 132, 186, 194, 1...","[(130, 132), (228, 229), (283, 284), (333, 335...",22,NaN
1889,Morgan_Chase=-2,Boeing=2,57,56,9,0.600000,0.157895,0.160714,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[16, 20, 28, 33, 42, 44, 52, 57, 65, 94, 104, ...","[(55, 57), (130, 132), (369, 370), (402, 404),...",15,NaN
1890,Morgan_Chase=-2,Altria=-1,57,182,16,0.761905,0.280702,0.087912,"[24, 33, 55, 66, 81, 107, 109, 130, 134, 170, ...","[17, 20, 36, 37, 43, 49, 50, 52, 54, 59, 62, 7...","[(81, 83), (107, 108), (130, 132), (170, 171),...",21,NaN


We search the best tradeoff between frequency and recall.
First, we will drop the rows that have either frequency or recall set as NaN because of a division by 0.
Then, we will compute the harmonic mean between confidence and recall and select the 10 best episodes.

The answer to question 2 is the first line of the output dataframe. For efficient trading, the other rules below should also be considered.

In [ ]:
import statistics #we need the harmonic mean

df_event_types['hMeanConfRec'] = df_event_types.apply(lambda row: statistics.harmonic_mean([row.confidence, row.recall]), axis=1)
df_event_types.nlargest(10, 'hMeanConfRec')

,X,Y,Xn,Yn,XYhead,frequency,confidence,recall,Xseq,Yseq,XYjoinSeq,L,hMeanConfRec
193,Boeing=-1,Boeing=1,269,300,119,0.798658,0.442379,0.396667,"[2, 7, 11, 12, 17, 18, 24, 34, 39, 53, 54, 62,...","[6, 8, 14, 23, 25, 29, 35, 37, 38, 46, 56, 60,...","[(7, 8), (12, 14), (24, 25), (34, 35), (54, 56...",149,0.418278
759,General_Motors=-1,Exxon_Mobil=1,266,263,108,0.755245,0.406015,0.410646,"[4, 6, 10, 12, 14, 15, 17, 26, 27, 30, 32, 33,...","[8, 13, 14, 16, 19, 20, 23, 27, 32, 41, 45, 57...","[(6, 8), (12, 13), (15, 16), (17, 19), (26, 27...",143,0.408318
1226,Microsoft=-1,Boeing=1,230,300,108,0.715232,0.469565,0.360000,"[8, 9, 13, 16, 19, 22, 32, 33, 35, 45, 53, 61,...","[6, 8, 14, 23, 25, 29, 35, 37, 38, 46, 56, 60,...","[(13, 14), (22, 23), (33, 35), (35, 37), (45, ...",151,0.407547
1221,Microsoft=-1,General_Motors=-1,230,266,100,0.709220,0.434783,0.375940,"[8, 9, 13, 16, 19, 22, 32, 33, 35, 45, 53, 61,...","[4, 6, 10, 12, 14, 15, 17, 26, 27, 30, 32, 33,...","[(9, 10), (13, 14), (16, 17), (32, 33), (33, 3...",141,0.403226
765,General_Motors=-1,General_Motors=1,266,244,102,0.744526,0.383459,0.418033,"[4, 6, 10, 12, 14, 15, 17, 26, 27, 30, 32, 33,...","[11, 16, 19, 21, 23, 24, 28, 31, 36, 40, 42, 4...","[(10, 11), (15, 16), (17, 19), (27, 28), (30, ...",137,0.400000
1362,AT&T=-1,Exxon_Mobil=1,227,263,97,0.697842,0.427313,0.368821,"[8, 9, 12, 13, 17, 22, 25, 34, 42, 46, 51, 54,...","[8, 13, 14, 16, 19, 20, 23, 27, 32, 41, 45, 57...","[(12, 13), (13, 14), (17, 19), (22, 23), (25, ...",139,0.395918
1455,Verizon=-1,American_Express=-1,250,230,95,0.766129,0.380000,0.413043,"[9, 13, 16, 25, 27, 33, 39, 40, 42, 51, 54, 62...","[12, 20, 24, 25, 29, 33, 40, 42, 59, 71, 76, 8...","[(27, 29), (39, 40), (40, 42), (69, 71), (84, ...",124,0.395833
953,Boeing=1,Exxon_Mobil=-1,300,228,103,0.725352,0.343333,0.451754,"[6, 8, 14, 23, 25, 29, 35, 37, 38, 46, 56, 60,...","[2, 5, 6, 9, 10, 11, 17, 18, 29, 30, 31, 38, 3...","[(8, 9), (29, 30), (37, 38), (38, 39), (46, 47...",142,0.390152
1450,Verizon=-1,AT&T=-1,250,227,93,0.699248,0.372000,0.409692,"[9, 13, 16, 25, 27, 33, 39, 40, 42, 51, 54, 62...","[8, 9, 12, 13, 17, 22, 25, 34, 42, 46, 51, 54,...","[(16, 17), (33, 34), (40, 42), (62, 63), (94, ...",133,0.389937
446,Intel=-1,General_Motors=-1,269,266,104,0.753623,0.386617,0.390977,"[2, 9, 10, 23, 25, 34, 35, 37, 38, 39, 49, 50,...","[4, 6, 10, 12, 14, 15, 17, 26, 27, 30, 32, 33,...","[(2, 4), (9, 10), (10, 12), (25, 26), (37, 38)...",138,0.388785
